# Experiment: classification with learned graph filters

We want to classify data by first extracting meaningful features from learned filters.

In [ ]:
import time
import numpy as np
import scipy.sparse, scipy.sparse.linalg, scipy.spatial.distance
from sklearn import datasets, linear_model
import matplotlib.pyplot as plt
%matplotlib inline
tol = 1e-10

## Dataset

* Two digits version of MNIST with N samples of each class.

In [ ]:
def mnist(a, b, N):
    """Prepare data for binary classification of MNIST."""
    mnist = datasets.fetch_mldata('MNIST original', data_home='.')

    assert N < min(sum(mnist.target==a), sum(mnist.target==b))
    M = mnist.data.shape[1]
    
    X = np.empty((N, 2, M))
    X[:,0,:] = mnist.data[mnist.target==a,:][:N,:]
    X[:,1,:] = mnist.data[mnist.target==b,:][:N,:]
    
    y = np.empty((N, 2))
    y[:,0] = -1
    y[:,1] = +1

    X.shape = 2*N, M
    y.shape = 2*N
    return X, y

X, y = mnist(5, 1, 1000)

N, M = X.shape
print('Dimensionality: N={} samples, M={} features'.format(N, M))

X -= 127.5
print('X in [{}, {}]'.format(np.min(X), np.max(X)))

def plot_digit(nn):
    m = int(np.sqrt(M))
    fig, axes = plt.subplots(1,len(nn), figsize=(15,5))
    for i, n in enumerate(nn):
        n = int(n)
        img = X[n,:]
        axes[i].imshow(img.reshape((m,m)))
        axes[i].set_title('Label: y = {:.0f}'.format(y[n]))

plot_digit([0, 1, 1e2, 1e2+1, 1e3, 1e3+1])

## Regularized least-square

### Loss and gradient

In [ ]:
def L(w, b=0):
    return np.linalg.norm(X @ w + b - y)**2 / N + tau * np.linalg.norm(w)**2

def dL(w, X, y):
    N = len(y)
    return 2 / N * X.T @ (X @ w - y) + 2 * tau * w

def print_perf(w):
    print('L({}) = {}'.format(w, L(eval(w))))
    print('|dL({})| = {}'.format(w, np.linalg.norm(dL(eval(w), X, y))))

### Reference: sklearn ridge regression

* With regularized data, the objective is the same with or without bias.

In [ ]:
tau = 1e3

clf = linear_model.Ridge(alpha=tau*N, fit_intercept=False)
clf.fit(X, y)
w_skl = clf.coef_

print('L(w_skl) = {}'.format(L(w_skl, clf.intercept_)))
print_perf('w_skl')

# Normalized data: intercept should be small.
print('bias: {}'.format(abs(np.mean(y - X @ w_skl))))

### Linear classifier

In [ ]:
w_d = np.linalg.inv(X.T @ X + tau * N * np.identity(M)) @ X.T @ y
print_perf('w_d')
np.testing.assert_allclose(w_d, w_skl, atol=1e-10)